# Practical machine learning and deep learning. Lab 5
## Competition
No competition for today



# Fine-tuning a model on a translation task
Today we will be finetunning T5(Text-To-Text Transfer Transformer) [model](https://github.com/google-research/t5x) on translation task. For this purpose we will be using [HuggingFace transformers](https://huggingface.co/docs/transformers/index) and [WMT16](https://huggingface.co/datasets/wmt16) dataset. 

In [ ]:
# installing huggingface libraries for dataset, models and metrics
!pip install datasets transformers[sentencepiece] sacrebleu

!pip install numpy==1.24.3

In [33]:
# Necessary inputs
import warnings

from datasets import load_dataset, load_metric
import transformers
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

warnings.filterwarnings('ignore')

## Selecting the model
For the example purpose we select as model checkpoint the smallest transformer in T5 family - `t5_small`. Other pre-trained models can be found [here](https://huggingface.co/docs/transformers/model_doc/t5#:~:text=T5%20comes%20in%20different%20sizes%3A).

In [34]:
# selecting model checkpoint
model_checkpoint = "t5-small"

## Loading the dataset

In [ ]:
# setting random seed for transformers library
transformers.set_seed(42)

# Load the WMT16 dataset
raw_datasets = load_dataset("wmt16", "de-en")

# Load the BLUE metric
metric = load_metric("sacrebleu")

## Dataset
Downloaded from HuggingFace dataset is a `DatasetDict`. It contains keys `["train", "validation", "test"]` - which represents a dataset splits

In [36]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4548885
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2169
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2999
    })
})

In [69]:
# samples from train dataset
raw_datasets["train"][:5]

{'translation': [{'de': 'Wiederaufnahme der Sitzungsperiode',
   'en': 'Resumption of the session'},
  {'de': 'Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.',
   'en': 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.'},
  {'de': 'Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.',
   'en': "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful."},
  {'de': 'Im Parlament besteht der Wunsch na

## Metric
[Sacrebleu](https://huggingface.co/spaces/evaluate-metric/sacrebleu) computes:
- `score`: BLEU score
- `counts`: list of counts of correct n-grams
- `totals`: list of counts of total n-grams
- `precisions`: list of precisions
- `bp`: Brevity penalty
- `sys_len`: cumulative sysem length
- `ref_len`: cumulative reference length

The main metric is [BLEU score](https://en.wikipedia.org/wiki/BLEU). BLEU (BiLingual Evaluation Understudy) is a metric for automatically evaluating machine-translated text. The BLEU score measures the similarity of the machine-translated text to a set of high quality reference translations.

The BLEU metric is calculates using [n-grams](https://en.wikipedia.org/wiki/N-gram).

In [70]:
# how to use sacrebleu and its purpose
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'exp'`. Possible values are:
        - `'none'`: no smoothing
        - `'floor'`: increment zero counts
        - `'add-k'`: increment num/deno

In [86]:
fake_preds = ["hello there", "general kenobi", "Can I get an A"]
fake_labels = [["hello there"], ["general kenobi"], ['Can I get a C']]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 45.59274666224604,
 'counts': [7, 4, 1, 0],
 'totals': [9, 6, 3, 2],
 'precisions': [77.77777777777777,
  66.66666666666667,
  33.333333333333336,
  25.0],
 'bp': 1.0,
 'sys_len': 9,
 'ref_len': 9}

## Preprocessing the data
As usual we will need to preprocess data and tokenize it before passing to model

In [42]:
from transformers import AutoTokenizer

# we will use autotokenizer for this purpose
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [43]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [44]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [45]:
# prefix for model input
prefix = "translate English to Deutsch:"

In [46]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "de"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [47]:
# example of preprocessing
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[13959, 1566, 12, 3, 18609, 10, 419, 4078, 102, 1575, 13, 8, 2363, 1], [13959, 1566, 12, 3, 18609, 10, 27, 15884, 4258, 26, 8, 2363, 13, 8, 1611, 12876, 19181, 1211, 29, 15, 26, 30, 1701, 1003, 1882, 5247, 6, 11, 27, 133, 114, 728, 541, 12, 1663, 25, 3, 9, 1095, 126, 215, 16, 8, 897, 24, 25, 2994, 3, 9, 8714, 15723, 1059, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[15158, 24860, 74, 11216, 425, 7, 4267, 32, 221, 1], [1674, 3, 49, 20635, 15, 67, 183, 17874, 6, 340, 11030, 17900, 1199, 5702, 1559, 15, 11216, 425, 7, 4267, 32, 221, 93, 3, 30604, 29, 13636, 7, 218, 1403, 3019, 7026, 6, 3, 25084, 2587, 18794, 7, 3532, 7756, 15, 674, 9242, 11621, 64, 3, 11950, 15, 6, 3, 26, 7118, 292, 11878, 16849, 8827, 5, 1]]}

In [ ]:
# for the example purpose we will crop the dataset and select first 5000 for train
# and 500 for validation and test
cropped_datasets = raw_datasets
cropped_datasets['train'] = raw_datasets['train'].select(range(5000))
cropped_datasets['validation'] = raw_datasets['validation'].select(range(500))
cropped_datasets['test'] = raw_datasets['test'].select(range(500))
tokenized_datasets = cropped_datasets.map(preprocess_function, batched=True)
tokenized_datasets['train'][0]

## Fine-tuning the model

In [49]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
# create a model for the pretrained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [50]:
# defining the parameters for training
batch_size = 32
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    report_to='tensorboard',
)

In [51]:
# instead of writing collate_fn function we will use DataCollatorForSeq2Seq
# simliarly it implements the batch creation for training
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [52]:
import numpy as np

# simple postprocessing for text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

# compute metrics function to pass to trainer
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [53]:
# instead of writing train loop we will use Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [54]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,1.321529,11.317800,17.656000
2,No log,1.318983,11.436100,17.628000
3,No log,1.318480,11.293400,17.628000
4,No log,1.318595,11.334700,17.634000
5,No log,1.318590,11.361100,17.626000
6,No log,1.318964,11.259700,17.624000
7,1.301300,1.319077,11.268900,17.622000
8,1.301300,1.319275,11.263800,17.626000
9,1.301300,1.319407,11.285900,17.618000
10,1.301300,1.319475,11.276600,17.614000


TrainOutput(global_step=790, training_loss=1.289421912084652, metrics={'train_runtime': 518.2799, 'train_samples_per_second': 96.473, 'train_steps_per_second': 1.524, 'total_flos': 1368531887456256.0, 'train_loss': 1.289421912084652, 'epoch': 10.0})

In [62]:
# saving model
trainer.save_model('best')

In [63]:
# loading the model and run inference for it
model = AutoModelForSeq2SeqLM.from_pretrained('best')
model.eval()
model.config.use_cache = False

In [64]:
def translate(model, inference_request, tokenizer=tokenizer):
    input_ids = tokenizer(inference_request, return_tensors="pt").input_ids
    outputs = model.generate(input_ids=input_ids)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True,temperature=0))

In [88]:
inference_request = prefix + 'Why did it take so long to train the model?'
translate(model, inference_request,tokenizer)

Warum hat es so lange dauern, das Modell zu trainieren?


In [89]:
inference_request = prefix +"My name is Wolfgang and I live in Berlin"
translate(model, inference_request,tokenizer)

Mein Name ist Wolfgang und ich lebe in Berlin.


In [92]:
inference_request = prefix + "Your assignment is hard. Start it today"
translate(model, inference_request,tokenizer)

Ihre Aufgabe ist schwierig, beginnen Sie sie heute
